In [39]:
import pandas as pd
from tpot import TPOTClassifier, TPOTRegressor
import time
import os
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [40]:
os.environ['KMP_DUPLICATE_LIB_OK']='True' # solves xgboost conflict 'OMP: Error #15: Initializing libiomp5.dylib, but found libiomp5.dylib already initialized.'

In [41]:
import multiprocessing
if __name__ == '__main__':
    multiprocessing.set_start_method('forkserver')
    # Note: need move import sklearn into main unless a RuntimeError (RuntimeError: context has already been set) will raise

RuntimeError: context has already been set

In [42]:
# Set the path to store the tpot pipelines
pipelines_path = 'tpot_pipelines'

In [43]:
data = pd.read_csv('pp_sample_tiny.csv')

In [44]:
imd = pd.read_csv('imd.csv')

* newhold'Y':1, 'N':0
* freehold 'F':1, 'L':0
* housetype'O': 0,'D': 1, 'S':2, 'T':3,'F':4

In [45]:
data.head()

,Unnamed: 0,Idx,Price,PC,PC_O,PC_1,Newold_OHE,freehold_OHE,Type_OHE,month,year
0,0,374492,286000,KT8 9AZ,KT8,9AZ,1,1.0,1,11,1995
1,1,531195,98950,HR8 1PP,HR8,1PP,0,1.0,1,10,1995
2,2,481861,86950,HP2 4UG,HP2,4UG,1,1.0,3,6,1995
3,3,651484,45000,WA11 8HD,WA11,8HD,0,0.0,2,5,1995
4,4,628854,33000,DH3 2DG,DH3,2DG,0,1.0,2,9,1995


In [46]:
len(data)

240000

In [47]:
len(imd), len(imd.loc[imd.LADC.isna()])

(2321, 0)

In [48]:
len(imd['PC'].unique())

2321

In [49]:
len(imd[imd['PC'].isin(data['PC'])])

71

In [50]:
imd.tail()

,LADC,index,PC,PC_O,PC_1
2316,E01033651,"11,504",M61 0EL,M61,0EL
2317,E01033651,"11,504",M99 1FF,M99,1FF
2318,E01033689,"5,499",BD98 8AA,BD98,8AA
2319,E01033748,"16,004",L73 6BT,L73,6BT
2320,E01033750,"20,066",L74 4BJ,L74,4BJ


In [51]:
dataset = pd.merge(data,imd, on = 'PC_O', how='right')#,  left_index=False, right_index=False, validate="many_to_one")

In [52]:
dataset = pd.merge(data, imd,on='PC_O', how='left')

In [53]:
len(dataset), len(data)

(240000, 240000)

In [54]:
dataset.tail()

,Unnamed: 0,Idx,Price,PC_x,PC_O,PC_1_x,Newold_OHE,freehold_OHE,Type_OHE,month,year,LADC,index,PC_y,PC_1_y
239995,239995,23547543,874115,WA1 1HQ,WA1,1HQ,0,1.0,0,3,2018,E01012478,"11,897",WA1 2DL,2DL
239996,239996,23144710,312000,BH12 3HH,BH12,3HH,0,1.0,1,5,2018,E01015379,"4,570",BH12 5AD,5AD
239997,239997,23862808,220000,NR33 8HZ,NR33,8HZ,0,1.0,2,10,2018,E01030233,"29,765",NR33 8AE,8AE
239998,239998,23733049,600000,CO4 5AX,CO4,5AX,0,1.0,1,1,2018,E01021648,"11,720",CO4 3BF,3BF
239999,239999,23396862,76000,SO15 5DD,SO15,5DD,0,0.0,4,3,2018,E01017136,"14,084",SO15 1DB,1DB


In [55]:
dataset['LADC'].replace(regex=True,inplace=True,to_replace=r'E0',value=r'')

In [56]:
dataset.loc[dataset.freehold_OHE.isna()]

,Unnamed: 0,Idx,Price,PC_x,PC_O,PC_1_x,Newold_OHE,freehold_OHE,Type_OHE,month,year,LADC,index,PC_y,PC_1_y
66087,66087,6365505,92000,TR7 1QP,TR7,1QP,0,NaN,3,1,2001,1019018,"22,039",TR7 2RL,2RL
68159,68159,6478171,204000,EN4 8DH,EN4,8DH,0,NaN,2,9,2001,1000115,"30,400",EN4 8LH,8LH
78118,78118,8333198,31000,OL1 3QF,OL1,3QF,0,NaN,2,2,2002,1005335,"24,858",OL1 2TG,2TG
78870,78870,7877068,150000,NW11 9DS,NW11,9DS,0,NaN,1,7,2002,1000135,"20,498",NW11 8AL,8AL


In [57]:
index_nan = dataset.loc[dataset.freehold_OHE.isna()].index #freehold_OHE
index_nan_index = dataset.loc[dataset.index.isna()].index

In [58]:
dataset.loc[dataset.Price.isna()]

,Unnamed: 0,Idx,Price,PC_x,PC_O,PC_1_x,Newold_OHE,freehold_OHE,Type_OHE,month,year,LADC,index,PC_y,PC_1_y


In [59]:
#dataset[dataset.LADC.isna()]

In [60]:
dataset_aux = dataset.copy()
dataset_aux = dataset.drop(index_nan)
dataset_aux = dataset.drop(index_nan_index)

len(dataset_aux)

240000

In [61]:
dataset_aux['index'].replace(regex=True,inplace=True,to_replace=r',',value=r'')

In [62]:
dataset_aux['freehold_OHE'] = pd.to_numeric(dataset_aux['freehold_OHE'])
dataset_aux['LADC'] = pd.to_numeric(dataset_aux['LADC'])
dataset_aux['index'] = pd.to_numeric(dataset_aux['index'])

In [63]:
# Test split
X_train, X_test, y_train, y_test = train_test_split(dataset_aux[['Newold_OHE','freehold_OHE', 'Type_OHE','index',
                                                                 'year']].values,\
                                                    dataset_aux['Price'].values,
                                                    train_size=0.75, test_size=0.25)
#['Newold_OHE', 'freehold_OHE', 'Type_OHE', 'year', 'month']

In [64]:
start_time = time.time()
pipeline_optimizer = TPOTRegressor(generations=2, population_size=20, verbosity=2)
pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))
pipeline_optimizer.export(os.path.join(pipelines_path,'tpot_exported_pipeline_index.py'))
time_taken = time.time() - start_time 
print(time_taken/60, 'minutes')

Imputing missing values in feature set


/Users/laraalegre/anaconda3/envs/conda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Generation 1 - Current best internal CV score: -920008928655.3258
Generation 2 - Current best internal CV score: -920008928655.3258

Best pipeline: ExtraTreesRegressor(input_matrix, bootstrap=False, max_features=0.55, min_samples_leaf=18, min_samples_split=5, n_estimators=100)
Imputing missing values in feature set
-232994316340.11935
43.5042686343193 minutes


In [66]:
from sklearn.ensemble import ExtraTreesRegressor

In [80]:
dataset_aux[dataset_aux[['Newold_OHE','freehold_OHE', 'Type_OHE','index','year']].isna()]

,Unnamed: 0,Idx,Price,PC_x,PC_O,PC_1_x,Newold_OHE,freehold_OHE,Type_OHE,month,year,LADC,index,PC_y,PC_1_y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
model = ExtraTreesRegressor(bootstrap=False, max_features=0.55, min_samples_leaf=18, min_samples_split=5, n_estimators=100)

model.fit(X_train, y_train)
results = model.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').